# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::382683543627:role/service-role/AmazonSageMaker-ExecutionRole-20230303T194309


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://object-detection-project-1/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

Cloning into 'docker/models'...


In [6]:
# build and push the docker image. This code can be commented after being ran once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection
Sending build context to Docker daemon    723MB
Step 1/17 : FROM tensorflow/tensorflow:2.9.0-gpu
2.9.0-gpu: Pulling from tensorflow/tensorflow

17ec1767: Pulling fs layer 
9ecd2bff: Pulling fs layer 
4ae53552: Pulling fs layer 
2d09b8c4: Pulling fs layer 
0d530989: Pulling fs layer 
81af025b: Pulling fs layer 
c129f45e: Pulling fs layer 
8fcb70c6: Pulling fs layer 
9aa4a247: Pulling fs layer 
3100c8d1: Pulling fs layer 
3a6b487b: Pulling fs layer 
e8773234: Pulling fs layer 
36c9476c: Pulling fs layer 
1b420cea: Pull complete 087kB/1.087kBBDigest: sha256:aa9f4a6a7debc976135702118aedfd0d72bf9e495af6ecfd5a31d9714e335426
Status: Downloaded newer image for tensorflow/tensorflow:2.9.0-gpu
 ---> c8d9ee2a0ff4


Removing intermediate container 00cdc47c44f7
 ---> 710b6c4d8177
Step 4/17 : RUN apt-key del 7fa2af80
 ---> Running in 97e81a7790d3
OK
Removing intermediate container 97e81a7790d3
 ---> 67c51772713e
Step 5/17 : RUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
 ---> Running in 656e7a1bc59b
Executing: /tmp/apt-key-gpghome.FiXwIsSD0V/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub'
gpg: key A4B469963BF863CC: "cudatools <cudatools@nvidia.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Removing intermediate container 656e7a1bc59b
 ---> 75aac5cb7a58
Step 6/17 : RUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/7fa2af80.pub
 ---> Running in 615665e46672
Exec

Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 less amd64 551-1ubuntu0.1 [123 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal/main amd64 libbsd0 amd64 0.10.0-1 [45.4 kB]
Get:14 http://archive.ubuntu.com/ubuntu focal/main amd64 python3-cffi-backend amd64 1.14.0-1build1 [68.7 kB]
Get:15 http://archive.ubuntu.com/ubuntu focal/main amd64 shared-mime-info amd64 1.15-1 [430 kB]
Get:16 http://archive.ubuntu.com/ubuntu focal/main amd64 ucf all 3.0038+nmu1 [51.6 kB]
Get:17 http://archive.ubuntu.com/ubuntu focal/main amd64 libcbor0.6 amd64 0.6.0-0ubuntu1 [21.1 kB]
Get:18 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdrm-common all 2.4.107-8ubuntu1~20.04.2 [5396 B]
Get:19 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libdrm2 amd64 2.4.107-8ubuntu1~20.04.2 [34.1 kB]
Get:20 http://archive.ubuntu.com/ubuntu focal/main amd64 libedit2 amd64 3.1-20191231-1 [87.0 kB]
Get:21 http://archive.ubuntu.com/ubuntu focal/main amd64 libfido2-1 amd64 1.3.1-1ubuntu2 [4

Get:94 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libwebp6 amd64 0.6.1-2ubuntu0.20.04.1 [185 kB]
Get:95 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libtiff5 amd64 4.1.0+git191117-2ubuntu0.20.04.7 [163 kB]
Get:96 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgdk-pixbuf2.0-common all 2.40.0+dfsg-3ubuntu0.4 [4592 B]
Get:97 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgdk-pixbuf2.0-0 amd64 2.40.0+dfsg-3ubuntu0.4 [168 kB]
Get:98 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgdk-pixbuf2.0-bin amd64 2.40.0+dfsg-3ubuntu0.4 [14.1 kB]
Get:99 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgles1 amd64 1.3.2-1~ubuntu0.20.04.2 [10.3 kB]
Get:100 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgles2 amd64 1.3.2-1~ubuntu0.20.04.2 [15.6 kB]
Get:101 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgles-dev amd64 1.3.2-1~ubuntu0.20.04.2 [47.9 kB]
Get:102 http://archive.ubuntu.com/ubuntu focal/

Selecting previously unselected package libx11-6:amd64.
Preparing to unpack .../014-libx11-6_2%3a1.6.9-2ubuntu1.2_amd64.deb ...
Unpacking libx11-6:amd64 (2:1.6.9-2ubuntu1.2) ...
Selecting previously unselected package libxext6:amd64.
Preparing to unpack .../015-libxext6_2%3a1.3.4-0ubuntu1_amd64.deb ...
Unpacking libxext6:amd64 (2:1.3.4-0ubuntu1) ...
Selecting previously unselected package libxmuu1:amd64.
Preparing to unpack .../016-libxmuu1_2%3a1.1.3-0ubuntu1_amd64.deb ...
Unpacking libxmuu1:amd64 (2:1.1.3-0ubuntu1) ...
Selecting previously unselected package openssh-client.
Preparing to unpack .../017-openssh-client_1%3a8.2p1-4ubuntu0.5_amd64.deb ...
Unpacking openssh-client (1:8.2p1-4ubuntu0.5) ...
Selecting previously unselected package wget.
Preparing to unpack .../018-wget_1.20.3-1ubuntu2_amd64.deb ...
Unpacking wget (1.20.3-1ubuntu2) ...
Selecting previously unselected package xauth.
Preparing to unpack .../019-xauth_1%3a1.1-0ubuntu1_amd64.deb ...
Unpacking xauth (1:1.1-0ubuntu1)

...

  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78543 sha256=6e34345948820bb2cb79205024fdfc5b1a1ff9fb4ae1405f26077542cab04067
  Stored in directory: /root/.cache/pip/wheels/07/35/78/e9004fa30578734db7f10e7a211605f3f0778d2bdde38a239d
  Created wheel for kaggle: filename=kaggle-1.5.13-py3-none-any.whl size=77714 sha256=841eafe9af8ee123aa5bf2565d97e21a56bab34183564cad6e7cc6b3d78085c3
  Stored in directory: /root/.cache/pip/wheels/e6/8e/67/e07554a720a493dc6b39b30488590ba92ed45448ad0134d253
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16166 sha256=816b73d8fbddee9d5afb6e00fc4e8aca6d1e13f64f097ae3284860efa5f8e726
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=205db96808d517834a988d7d5ed0026e94c7b8820b1aa43cf5b3af99cd63068c
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e32685

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.5/618.5 kB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.8 MB/s eta 0:00:00
  Created wheel for sagemaker-training: filename=sagemaker_training-4.4.7-cp38-cp38-linux_x86_64.whl size=92185 sha256=7d5d8c156c40780226faff172f5d71af701f6143b9de2b9d2d5b544d3cf6fe86
  Stored in directory: /root/.cache/pip/wheels/36/88/61/d8d5a84bd7e2d34c74598773c949ab0723b20f7f02b8fc7a49
  Created wheel for inotify_simple: filename=inotify_simple-1.2

fcbe1ae8: Pushing    196MB/514.2MB574c12e: Pushing  502.7MB/1.455GBPushing  605.1MB/3.353GBfae0f58c: Pushing  1.082GB/3.681GBPushing  1.191GB/3.353GBPushing  539.4MB/3.681GBfae0f58c: Pushing  2.173GB/3.681GBPushing  1.506GB/3.353GBPushing  1.687GB/3.681GB

c0dfae2: Pushing   3.37GB/3.681GBPushing  2.464GB/3.681GB

fae0f58c: Pushed   3.691GB/3.681GB20230303142333: digest: sha256:fa8a79e73648eb82ebaafe1582d1b133409ec58a25378c10b9149c1fbbc96268 size: 5782
Saving ECR image URI into ecr_image_fullname.txt


To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [7]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

382683543627.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20230303142333


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [10]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet101_v1_640x640_coco17_tpu-8/checkpoint

mkdir: cannot create directory ‘/tmp/checkpoint’: File exists
mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2023-03-03 14:43:16--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.163.128, 2607:f8b0:4004:c1b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.163.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353643040 (337M) [application/x-tar]
Saving to: ‘/tmp/efficientdet.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 12.1M 28s
    50K .......... .......... .......... .......... ..........  0% 5.19M 46s
   100K .......... .......... .......... .......... ..........  0% 16.1M 38s
   150K .......... .......... .......... .......... ..........  0% 5.39M 44s
   200K .......... .......... .......... .......... ..........  0%  266M 36s
   250K

  4950K .......... .......... .......... .......... ..........  1%  377M 12s
  5000K .......... .......... .......... .......... ..........  1%  348M 11s
  5050K .......... .......... .......... .......... ..........  1%  269M 11s
  5100K .......... .......... .......... .......... ..........  1%  117M 11s
  5150K .......... .......... .......... .......... ..........  1% 43.4M 11s
  5200K .......... .......... .......... .......... ..........  1%  309M 11s
  5250K .......... .......... .......... .......... ..........  1% 50.9M 11s
  5300K .......... .......... .......... .......... ..........  1% 52.7M 11s
  5350K .......... .......... .......... .......... ..........  1% 6.10M 11s
  5400K .......... .......... .......... .......... ..........  1%  159M 11s
  5450K .......... .......... .......... .......... ..........  1%  239M 11s
  5500K .......... .......... .......... .......... ..........  1%  369M 11s
  5550K .......... .......... .......... .......... ..........  1% 22.6M 11s

 10350K .......... .......... .......... .......... ..........  3%  158M 9s
 10400K .......... .......... .......... .......... ..........  3%  323M 9s
 10450K .......... .......... .......... .......... ..........  3% 23.3M 9s
 10500K .......... .......... .......... .......... ..........  3%  238M 9s
 10550K .......... .......... .......... .......... ..........  3%  317M 9s
 10600K .......... .......... .......... .......... ..........  3%  342M 9s
 10650K .......... .......... .......... .......... ..........  3%  171M 9s
 10700K .......... .......... .......... .......... ..........  3%  269M 8s
 10750K .......... .......... .......... .......... ..........  3%  281M 8s
 10800K .......... .......... .......... .......... ..........  3%  349M 8s
 10850K .......... .......... .......... .......... ..........  3%  348M 8s
 10900K .......... .......... .......... .......... ..........  3%  147M 8s
 10950K .......... .......... .......... .......... ..........  3%  331M 8s
 11000K ....

 15750K .......... .......... .......... .......... ..........  4%  318M 8s
 15800K .......... .......... .......... .......... ..........  4%  316M 8s
 15850K .......... .......... .......... .......... ..........  4%  347M 8s
 15900K .......... .......... .......... .......... ..........  4% 12.1M 8s
 15950K .......... .......... .......... .......... ..........  4%  263M 8s
 16000K .......... .......... .......... .......... ..........  4% 9.26M 8s
 16050K .......... .......... .......... .......... ..........  4%  338M 8s
 16100K .......... .......... .......... .......... ..........  4%  242M 8s
 16150K .......... .......... .......... .......... ..........  4%  343M 8s
 16200K .......... .......... .......... .......... ..........  4%  318M 8s
 16250K .......... .......... .......... .......... ..........  4%  309M 8s
 16300K .......... .......... .......... .......... ..........  4% 13.5M 8s
 16350K .......... .......... .......... .......... ..........  4%  252M 8s
 16400K ....

 21150K .......... .......... .......... .......... ..........  6%  235M 7s
 21200K .......... .......... .......... .......... ..........  6%  129M 7s
 21250K .......... .......... .......... .......... ..........  6%  191M 7s
 21300K .......... .......... .......... .......... ..........  6%  306M 7s
 21350K .......... .......... .......... .......... ..........  6%  185M 7s
 21400K .......... .......... .......... .......... ..........  6%  326M 7s
 21450K .......... .......... .......... .......... ..........  6% 84.5M 7s
 21500K .......... .......... .......... .......... ..........  6%  241M 7s
 21550K .......... .......... .......... .......... ..........  6%  205M 7s
 21600K .......... .......... .......... .......... ..........  6%  341M 7s
 21650K .......... .......... .......... .......... ..........  6%  167M 7s
 21700K .......... .......... .......... .......... ..........  6%  291M 7s
 21750K .......... .......... .......... .......... ..........  6%  158M 7s
 21800K ....

...

296550K .......... .......... .......... .......... .......... 85%  275M 1s
296600K .......... .......... .......... .......... .......... 85%  307M 1s
296650K .......... .......... .......... .......... .......... 85%  368M 1s
296700K .......... .......... .......... .......... .......... 85%  342M 1s
296750K .......... .......... .......... .......... .......... 85%  273M 1s
296800K .......... .......... .......... .......... .......... 85%  263M 1s
296850K .......... .......... .......... .......... .......... 85%  304M 1s
296900K .......... .......... .......... .......... .......... 85%  292M 1s
296950K .......... .......... .......... .......... .......... 85%  318M 1s
297000K .......... .......... .......... .......... .......... 86%  297M 1s
297050K .......... .......... .......... .......... .......... 86%  317M 1s
297100K .......... .......... .......... .......... .......... 86%  353M 1s
297150K .......... .......... .......... .......... .......... 86%  308M 1s
297200K ....

301950K .......... .......... .......... .......... .......... 87%  110M 1s
302000K .......... .......... .......... .......... .......... 87%  269M 1s
302050K .......... .......... .......... .......... .......... 87%  342M 1s
302100K .......... .......... .......... .......... .......... 87%  374M 1s
302150K .......... .......... .......... .......... .......... 87%  241M 1s
302200K .......... .......... .......... .......... .......... 87%  281M 1s
302250K .......... .......... .......... .......... .......... 87%  364M 1s
302300K .......... .......... .......... .......... .......... 87%  142M 1s
302350K .......... .......... .......... .......... .......... 87%  331M 1s
302400K .......... .......... .......... .......... .......... 87%  283M 1s
302450K .......... .......... .......... .......... .......... 87%  374M 1s
302500K .......... .......... .......... .......... .......... 87%  154M 1s
302550K .......... .......... .......... .......... .......... 87%  313M 1s
302600K ....

307350K .......... .......... .......... .......... .......... 89%  299M 1s
307400K .......... .......... .......... .......... .......... 89%  280M 1s
307450K .......... .......... .......... .......... .......... 89%  317M 1s
307500K .......... .......... .......... .......... .......... 89% 7.21M 1s
307550K .......... .......... .......... .......... .......... 89%  203M 1s
307600K .......... .......... .......... .......... .......... 89%  263M 1s
307650K .......... .......... .......... .......... .......... 89%  351M 1s
307700K .......... .......... .......... .......... .......... 89%  296M 1s
307750K .......... .......... .......... .......... .......... 89%  333M 1s
307800K .......... .......... .......... .......... .......... 89%  281M 1s
307850K .......... .......... .......... .......... .......... 89%  362M 1s
307900K .......... .......... .......... .......... .......... 89%  362M 1s
307950K .......... .......... .......... .......... .......... 89%  299M 1s
308000K ....

312750K .......... .......... .......... .......... .......... 90%  372M 0s
312800K .......... .......... .......... .......... .......... 90%  311M 0s
312850K .......... .......... .......... .......... .......... 90%  322M 0s
312900K .......... .......... .......... .......... .......... 90%  364M 0s
312950K .......... .......... .......... .......... .......... 90%  322M 0s
313000K .......... .......... .......... .......... .......... 90%  278M 0s
313050K .......... .......... .......... .......... .......... 90%  342M 0s
313100K .......... .......... .......... .......... .......... 90%  346M 0s
313150K .......... .......... .......... .......... .......... 90%  302M 0s
313200K .......... .......... .......... .......... .......... 90%  264M 0s
313250K .......... .......... .......... .......... .......... 90%  331M 0s
313300K .......... .......... .......... .......... .......... 90%  342M 0s
313350K .......... .......... .......... .......... .......... 90%  346M 0s
313400K ....

318150K .......... .......... .......... .......... .......... 92%  324M 0s
318200K .......... .......... .......... .......... .......... 92%  266M 0s
318250K .......... .......... .......... .......... .......... 92%  359M 0s
318300K .......... .......... .......... .......... .......... 92%  327M 0s
318350K .......... .......... .......... .......... .......... 92%  299M 0s
318400K .......... .......... .......... .......... .......... 92%  264M 0s
318450K .......... .......... .......... .......... .......... 92%  273M 0s
318500K .......... .......... .......... .......... .......... 92%  309M 0s
318550K .......... .......... .......... .......... .......... 92%  340M 0s
318600K .......... .......... .......... .......... .......... 92%  275M 0s
318650K .......... .......... .......... .......... .......... 92%  357M 0s
318700K .......... .......... .......... .......... .......... 92%  291M 0s
318750K .......... .......... .......... .......... .......... 92%  337M 0s
318800K ....

## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [11]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2023-03-03-14-46-21-233


2023-03-03 14:46:34 Starting - Starting the training job...
2023-03-03 14:47:01 Starting - Preparing the instances for training......
2023-03-03 14:47:58 Downloading - Downloading input data...
2023-03-03 14:48:23 Training - Downloading the training image...............
2023-03-03 14:50:39 Training - Training image download completed. Training in progress..2023-03-03 14:51:07,662 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-03 14:51:07,696 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-03 14:51:07,729 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-03 14:51:07,743 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "current

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0303 14:51:15.778331 140335365093184 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 2000
I0303 14:51:15.782121 140335365093184 config_util.py:552] Maybe overwriting train_steps: 2000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0303 14:51:15.782254 140335365093184 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0303 14:51:15.824308 140335365093184 deprecation.py:350] From /usr/local/lib/python3.8/dist-packages/object_detection/model_lib_v2.py:563: StrategyBase.experimental_distribute_datasets_from_function (from tensorflow.python.distribute.distribute_lib) is deprecated and will be removed in a future version.
Instructions for updating:
rename to distribute_datasets_from_function
INFO

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0303 14:52:17.927998 140335365093184 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0303 14:52:17.931385 140335365093184 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
I0303 14:52:17.932575 140335365093184 cross_device_ops.py:616] Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Red

INFO:tensorflow:Step 600 per-step time 0.252s
I0303 14:56:11.448497 140335365093184 model_lib_v2.py:705] Step 600 per-step time 0.252s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.30619407,
 'Loss/BoxClassifierLoss/localization_loss': 0.52346087,
 'Loss/RPNLoss/localization_loss': 0.9266556,
 'Loss/RPNLoss/objectness_loss': 0.22310337,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.979414,
 'learning_rate': 0.019960001}
I0303 14:56:11.448999 140335365093184 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.30619407,
 'Loss/BoxClassifierLoss/localization_loss': 0.52346087,
 'Loss/RPNLoss/localization_loss': 0.9266556,
 'Loss/RPNLoss/objectness_loss': 0.22310337,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.979414,
 'learning_rate': 0.019960001}
INFO:tensorflow:Step 700 per-step time 0.251s
I0303 14:56:36.542958 140335365093184 model_lib_v2.py:705] Step 700 per-step time 0.251s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classifica

INFO:tensorflow:Step 1600 per-step time 0.252s
I0303 15:00:24.996656 140335365093184 model_lib_v2.py:705] Step 1600 per-step time 0.252s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification_loss': 0.19040431,
 'Loss/BoxClassifierLoss/localization_loss': 0.34120765,
 'Loss/RPNLoss/localization_loss': 0.5777699,
 'Loss/RPNLoss/objectness_loss': 0.13132404,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.240706,
 'learning_rate': 0.05156}
I0303 15:00:24.997143 140335365093184 model_lib_v2.py:708] {'Loss/BoxClassifierLoss/classification_loss': 0.19040431,
 'Loss/BoxClassifierLoss/localization_loss': 0.34120765,
 'Loss/RPNLoss/localization_loss': 0.5777699,
 'Loss/RPNLoss/objectness_loss': 0.13132404,
 'Loss/regularization_loss': 0.0,
 'Loss/total_loss': 1.240706,
 'learning_rate': 0.05156}
INFO:tensorflow:Step 1700 per-step time 0.248s
I0303 15:00:49.841421 140335365093184 model_lib_v2.py:705] Step 1700 per-step time 0.248s
INFO:tensorflow:{'Loss/BoxClassifierLoss/classification

/usr/local/lib/python3.8/dist-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
INFO:tensorflow:depth of additional conv before box predictor: 0
I0303 15:02:34.989556 140226917488448 convolutional_keras_box_predictor.py:152] depth of additional conv before box predictor: 0
Instructions for updating:
Use ref() instead.
W0303 15:02:42.179639 140226917488448 deprecation.py:350] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:459: Tensor.experimental_ref (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use ref() instead.
Instructions for updating:
Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.
See `tf.n

INFO:tensorflow:Waiting for new checkpoint at /opt/training
I0303 15:07:24.790893 140226917488448 checkpoint_utils.py:140] Waiting for new checkpoint at /opt/training
INFO:tensorflow:Timed-out waiting for a checkpoint.
I0303 15:07:33.806902 140226917488448 checkpoint_utils.py:203] Timed-out waiting for a checkpoint.
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=20.10s).
Accumulating evaluation results...
DONE (t=0.48s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.030
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.067
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.101
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.241
 Ave

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the intial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your writeup), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

## Write-up
In this experiment, I tried Faster RCNN model V101 with aspect ratio 640x640. This architecture is depicted in figure below and is a very good detection architecture.
I ran it with 30000 steps keeping batch size of 4.
![Faster RCNN](../data/img-1.jpg)

The loss definitely improved on using it as we can see in the cells output above.
The loss has a decreasing tendency. However, the mAP (mean average precision) doesn’t increase as the loss decreases.
I think this is because of the small number of training images which leads to overfitting of the model.
Eventually I tried Resnet 50 model architecture as well and got better performance by applying 
data augmentations like horizontal flip and adjust brightness, pipeline config files are placed inside source_dir directory.
The final video and images are available inside data directory of base project folder.
